In [ ]:
#pip install findspark

#/usr/local/opt/apache-spark/libexec/sbin
#./start-thriftserver.sh


In [1]:
import findspark
findspark.init()

In [2]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("spark").config('spark.driver.memory', '32g').config("hive.server2.thrift.port", 10000).config("spark.sql.hive.thriftServer.singleSession", True).enableHiveSupport().getOrCreate()


22/03/30 17:39:12 WARN Utils: Your hostname, Ewins-Mac-Hex.local resolves to a loopback address: 127.0.0.1; using 192.168.0.47 instead (on interface en0)
22/03/30 17:39:12 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/03/30 17:39:13 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
from pyspark.sql.types import StructType, StructField
from pyspark.sql.types import DoubleType, IntegerType, StringType,TimestampType
from pyspark.sql.types import DateType
from pyspark.sql.functions import *

schema = StructType([
    StructField("ID", IntegerType()),
    StructField("Speed", DoubleType()),
    StructField("TravelTime", IntegerType()),
    StructField("Status", IntegerType()),
    StructField("timedate", StringType()),
    StructField("LinkId", IntegerType()),
    StructField("LinkPoints", StringType()),
    StructField("EncodedLinkPoints", StringType()),
    StructField("EncodedPolyLineLvls", StringType()),
    StructField("Owner", StringType()),
    StructField("TranscomId", IntegerType()),
    StructField("Borough", StringType()),
    StructField("Link_Name", StringType())
])

schema1 = "ID bigint, Speed float, TravelTime int, Status int, TimeDate string, LinkId int, LinkPoints string, EncodedLinkPoints string, EncodedPolyLineLvls string, Owner string, TranscomId int, Borough string,Link_Name string"
df2 = spark.read.format("csv").option("header", "true").schema(schema).load("file:/Users/minoseah629/Downloads/DOT_Traffic_Speeds_NBE.csv")
df2 = df2.withColumn("timedate", to_timestamp("timedate", 'MM/dd/yyyy hh:mm:ss a'))
df2 = df2.na.drop()
df2 = df2.drop(*("Status","EncodedLinkPoints","EncodedPolyLineLvls","TranscomId"))
df2.printSchema()
df2.createOrReplaceTempView("dataset")
# .option("inferSchema", "true")

root
 |-- ID: integer (nullable = true)
 |-- Speed: double (nullable = true)
 |-- TravelTime: integer (nullable = true)
 |-- timedate: timestamp (nullable = true)
 |-- LinkId: integer (nullable = true)
 |-- LinkPoints: string (nullable = true)
 |-- Owner: string (nullable = true)
 |-- Borough: string (nullable = true)
 |-- Link_Name: string (nullable = true)



In [59]:
avg = df2.agg({"speed": "avg"}).alias('avg').first().asDict()


22/03/30 21:09:15 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: ID, SPEED, TRAVEL_TIME, STATUS, DATA_AS_OF, LINK_ID, LINK_POINTS, ENCODED_POLY_LINE, ENCODED_POLY_LINE_LVLS, OWNER, TRANSCOM_ID, BOROUGH, LINK_NAME
 Schema: ID, Speed, TravelTime, Status, timedate, LinkId, LinkPoints, EncodedLinkPoints, EncodedPolyLineLvls, Owner, TranscomId, Borough, Link_Name
Expected: TravelTime but found: TRAVEL_TIME
CSV file: file:///Users/minoseah629/Downloads/DOT_Traffic_Speeds_NBE.csv


{'avg(speed)': 34.99334896089561}

In [65]:
display(avg['avg(speed)'])

34.99334896089561

In [1]:
avgString = str(avg['avg(speed)'])
speed = "case " \
     +"when ("+avgString+"/2) > speed and speed < "+avgString+"  then 0 "\
     +"else 1 end "
     # +"when speed > "+str(avg['avg(speed)'])+" then 1 "\
travel = "(traveltime/60) "
columns = "id, speed,year(timedate),month(timedate),day(timedate),hour(timedate),minute(timedate), second(timedate), "+travel
fromsql = " from dataset"
command = "select "+columns+ "as `traveltime in minutes` , "+speed +"as `speed group` "+ fromsql + " group by "+ columns
display(command)
data2022 = spark.sql(command)
data2022.show()

NameError: name 'avg' is not defined

In [79]:
data2022.write.format('csv').mode('overwrite').save('file:/Users/minoseah629/Desktop/ait614/speedgroups.csv')


22/03/30 21:43:09 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: ID, SPEED, TRAVEL_TIME, STATUS, DATA_AS_OF, LINK_ID, LINK_POINTS, ENCODED_POLY_LINE, ENCODED_POLY_LINE_LVLS, OWNER, TRANSCOM_ID, BOROUGH, LINK_NAME
 Schema: ID, Speed, TravelTime, Status, timedate, LinkId, LinkPoints, EncodedLinkPoints, EncodedPolyLineLvls, Owner, TranscomId, Borough, Link_Name
Expected: TravelTime but found: TRAVEL_TIME
CSV file: file:///Users/minoseah629/Downloads/DOT_Traffic_Speeds_NBE.csv


In [6]:
data2022.count()

22/03/22 23:33:18 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: ID, SPEED, TRAVEL_TIME, STATUS, DATA_AS_OF, LINK_ID, LINK_POINTS, ENCODED_POLY_LINE, ENCODED_POLY_LINE_LVLS, OWNER, TRANSCOM_ID, BOROUGH, LINK_NAME
 Schema: ID, Speed, TravelTime, Status, timedate, LinkId, LinkPoints, EncodedLinkPoints, EncodedPolyLineLvls, Owner, TranscomId, Borough, Link_Name
Expected: TravelTime but found: TRAVEL_TIME
CSV file: file:///Users/minoseah629/Downloads/DOT_Traffic_Speeds_NBE.csv


1893262

In [3]:
spark.stop()